In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import datetime as dt
import requests
import numpy as np
import pandas as pd

In [30]:
artarr = np.zeros((70, 4), dtype='>U60')
artarr

array([['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],
       ['', '', '', ''],


In [41]:
def crawlArtistFeatures(url):
    res = requests.get(url, headers={"User-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0 rv:11.0) like Gecko"})
    res.raise_for_status()
    
    driver = webdriver.Chrome()
    driver.get(url)
    for syllable in range(1, 15):
        for i in range(1, 6):
            click = driver.find_element_by_css_selector('#conts > div.wrap_finder.atist > dl > dd > div.wrap_btn_atist.first_child > button:nth-child('+str(syllable)+')')
            click.click()

            driver.maximize_window() 
            driver.implicitly_wait(10)



            artistClick = driver.find_element_by_css_selector('#pageList > div.list_atist12.d_artist_list > ul > li:nth-child('+ str(i) +') > div > div > dl > dt > a')
            artistClick.click()

            driver.implicitly_wait(10)

            detailinfo = driver.find_element_by_css_selector('#conts > div.wrap_tab_atist > ul > li:nth-child(2) > a')
            detailinfo.click()


            soup = BeautifulSoup(driver.page_source, 'html.parser')

            artist = soup.select('#conts > div.wrap_dtl_atist > div > div.wrap_atist_info > p')
            features_dt = soup.select('#conts > div.section_atistinfo03 > dl > dt')
            features = soup.select('#conts > div.section_atistinfo03 > dl > dd')
    #         print(features_dt)
    #         print(features)
    #         print(artist[0].text.strip().replace("아티스트명",""))
            artistproc1 = artist[0].text.strip().replace("아티스트명","")
            artistproc2 = artistproc1.replace(u"\xa0", u" ")
            artarr[((syllable-1)*5) + i-1, 0] = artistproc2

            j = 1
            if features_dt[0].text.strip() == "데뷔":
                for k in range(1, 4):
                    feature = features[k].text.strip().split('\n')
                    artarr[((syllable-1)*5) + i-1, j] = ''.join(feature).replace("\t", "")
                    j = j + 1
            elif features_dt[0].text.strip() == "활동년대":
                 for k in range(0, 3):
                    feature = features[k].text.strip().split('\n')
                    artarr[((syllable-1)*5) + i-1, j] = ''.join(feature).replace("\t", "")
                    j = j + 1

            driver.back()
            driver.back()

    driver.quit()

crawlArtistFeatures('https://www.melon.com/artistplus/finder/index.htm')

In [42]:
artarr

array([['기리보이 (홍시영)', '2010 년대', '솔로|남성',
        '발라드, 댄스, 랩/힙합, R&B/Soul, 인디음악'],
       ['김필', '2010 년대', '솔로|남성',
        '발라드, 댄스, R&B/Soul, 인디음악, 록/메탈, 포크/블루스'],
       ['김동률', '1990, 2000, 2010 년대', '솔로|남성', '발라드'],
       ['김하온 (HAON)', '2010 년대', '솔로|남성', '발라드, 랩/힙합'],
       ['김소유 (김소희)', '2010 년대', '솔로|여성', '성인가요'],
       ['닐로(Nilo)', '2010 년대', '솔로|남성', '발라드'],
       ['뉴이스트', '2010 년대', '2012', '그룹|남성'],
       ['나플라(nafla)', '2010 년대', '솔로|남성', '랩/힙합, 인디음악'],
       ['넬 (NELL)', '2000, 2010 년대', '1999', '그룹|남성'],
       ['남도현', '2010 년대', '솔로|남성', 'X1 (엑스원)'],
       ['다비치', '2000, 2010 년대', '2008', '그룹|여성'],
       ['다이나믹 듀오', '2000, 2010 년대', '2003', '그룹|남성'],
       ['더보이즈(THE BOYZ)', '2010 년대', '그룹|남성',
        '발라드, 댄스, R&B/Soul, 일렉트로니카'],
       ['디오 (D.O.)', '2010 년대', '솔로|남성', '발라드, R&B/Soul'],
       ['디에이드 (The Ade)', '2010 년대', '그룹|혼성', '발라드, 인디음악, R&B/Soul, 포크'],
       ['롤러 코스터', '1990, 2000 년대', '1999', '그룹|혼성'],
       ['릴러말즈 (Leellamarz)', '2010 년대', '솔로|